### SlideRule API

SlideRule is an on-demand science data processing service that runs in the cloud and responds to REST API calls to process and return science results.

SlideRule can be accessed by any http client (e.g. curl) by making GET and POST requests to the SlideRule service. For the purposes of this document, all requests to SlideRule will originate from a Python script using Python's requests module.

This notebook uses Jupyter widgets to set parameters for using the SlideRule API.
The widgets can be installed as described below.  
```bash
pip3 install --user ipywidgets
jupyter nbextension enable --py --user widgetsnbextension
jupyter-notebook
```

#### Load necessary packages

In [ ]:
from sliderule import icesat2
import numpy as np
import matplotlib.pyplot as plt
import cartopy
import ipywidgets as widgets
import ipyleaflet as leaflet
#-- autoreload
%load_ext autoreload
%autoreload 2

#### Configure SlideRule

In [ ]:
# set the url for the sliderule service.
icesat2.init("icesat2sliderule.org", False)

In [ ]:
# calculate centroid of polygon
def centroid(x,y):
    npts = len(x)
    area,cx,cy = (0.0,0.0,0.0)
    for i in range(npts-1):
        SA = x[i]*y[i+1] - x[i+1]*y[i]
        area += SA
        cx += (x[i] + x[i+1])*SA
        cy += (y[i] + y[i+1])*SA
    cx /= 3.0*area
    cy /= 3.0*area
    return (cx,cy)

# determine if polygon winding is counter-clockwise
def winding(x,y):
    npts = len(x)
    wind = np.sum([(x[i+1] - x[i])*(y[i+1] + y[i]) for i in range(npts - 1)])
    return wind

In [ ]:
# create ipyleaflet map centered on grand mesa
m = leaflet.Map(center=(39,-108), zoom=9, basemap=leaflet.basemaps.Esri.WorldTopoMap)
# add control for zoom
zoom_slider = widgets.IntSlider(description='Zoom level:', min=0, max=15, value=9)
widgets.jslink((zoom_slider, 'value'), (m, 'zoom'))
zoom_control =  leaflet.WidgetControl(widget=zoom_slider, position='topright')
m.add_control(zoom_control)
# add control for drawing polygons or bounding boxes
draw_control = leaflet.DrawControl(polyline={},circlemarker={})
draw_control.rectangle = dict(shapeOptions={'color':'green','fill_color':'green'})
draw_control.polygon = dict(shapeOptions={'color':'green','fill_color':'green'})

# An empty list for storing drawing geometries
regions = []
# keep track of rectangles and polygons drawn on map
def handle_draw(self, action, geo_json):
    lon,lat = np.transpose(geo_json['geometry']['coordinates'])
    cx,cy = centroid(lon,lat)
    wind = winding(lon,lat)
    # set winding to counter-clockwise
    if (wind > 0):
        lon = lon[::-1]
        lat = lat[::-1]
    # create sliderule region from list
    region = [{'lon':ln,'lat':lt} for ln,lt in np.c_[lon,lat]]
    # append coordinates to list
    if (action == 'created'):
        regions.append(region)
    elif (action == 'deleted'):
        regions.remove(region)
    return regions
draw_control.on_draw(handle_draw)
m.add_control(draw_control)
m

#### Set options for making science data processing requests to SlideRule

In [ ]:
# dropdown menu for setting asset
AssetDropdown = widgets.Dropdown(
    options=['atlas-local', 'atlas-s3'],
    value='atlas-s3',
    description='Asset:',
    disabled=False,
)

# dropdown menu for setting data release
ReleaseDropdown = widgets.Dropdown(
    options=['003', '004'],
    value='004',
    description='Release:',
    disabled=False,
)

# slider for setting length of ATL06 segment in meters
LengthSlider = widgets.IntSlider(
    value=40,
    min=5,
    max=200,
    step=10,
    description='Length:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# slider for setting step distance for successive segments in meters
StepSlider = widgets.IntSlider(
    value=20,
    min=5,
    max=200,
    step=5,
    description='Step:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# slider for setting confidence level for PE selection
# eventually would be good to switch this to a IntRangeSlider with value=[0,4]
ConfSlider = widgets.IntSlider(
    value=4,
    min=0,
    max=4,
    step=1,
    description='Confidence:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# slider for setting maximum number of iterations
# (not including initial least-squares-fit selection)
IterationSlider = widgets.IntSlider(
    value=1,
    min=0,
    max=20,
    step=1,
    description='Iterations:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# slider for setting minimum along track spread 
SpreadSlider = widgets.FloatSlider(
    value=20,
    min=1,
    max=100,
    step=0.1,
    description='Spread:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='0.1f'
)
# slider for setting minimum photon event (PE) count
CountSlider = widgets.IntSlider(
    value=10,
    min=1,
    max=50,
    step=1,
    description='PE Count:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# slider for setting minimum height of PE window in meters
WindowSlider = widgets.FloatSlider(
    value=3,
    min=0.5,
    max=10,
    step=0.1,
    description='Window:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='0.1f'
)

# slider for setting maximum robust dispersion in meters
SigmaSlider = widgets.FloatSlider(
    value=5,
    min=1,
    max=10,
    step=0.1,
    description='Sigma:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='0.1f'
)

# display widgets for setting SlideRule parameters
widgets.VBox([
    AssetDropdown,
    ReleaseDropdown,
    LengthSlider,
    StepSlider,
    ConfSlider,
    IterationSlider,
    SpreadSlider,
    CountSlider,
    WindowSlider,
    SigmaSlider
])

#### Build request and transmit request

In [ ]:
# sliderule asset and data release
asset = AssetDropdown.value
release = ReleaseDropdown.value

# build sliderule parameters
parms = {
    # polygon from map
    "poly": regions[0],
    # surface type: 0-land, 1-ocean, 2-sea ice, 3-land ice, 4-inland water (default: 3)
    "srt": icesat2.SRT_LAND,
    # length of ATL06-SR segment in meters (default: 40.0)
    "len": LengthSlider.value,
    # step distance for successive ATL06-SR segments in meters (default: 20.0)
    "res": StepSlider.value,
    # confidence level for PE selection (default: 4)
    "cnf": ConfSlider.value,
    # maximum iterations, not including initial least-squares-fit selection (default: 1)
    "maxi": IterationSlider.value,
    # minimum along track spread (default: 20.0)
    "ats": SpreadSlider.value,
    # minimum PE count (default: 10)
    "cnt": CountSlider.value,
    # minimum height of PE window in meters (default: 3.0)
    "H_min_win": WindowSlider.value,
    # maximum robust dispersion in meters (default: 5.0)
    "sigma_r_max": SigmaSlider.value
}

# make the request to the SlideRule (ATL06-SR) endpoint
# and pass it the request parameters to request ATL06 Data
gdf = icesat2.atl06p(parms, asset, version=release)

#### Create output plots

In [ ]:
# output map of resource location
f1, ax1 = plt.subplots(num=1, nrows=1, ncols=1, figsize=(10,6),
    subplot_kw=dict(projection=cartopy.crs.PlateCarree()))
ax1.plot(gdf.geometry.x,gdf.geometry.y,'r.',ms=1,transform=cartopy.crs.Geodetic())
ax1.set_extent((-180,180,-90,90),crs=cartopy.crs.PlateCarree())
# add coastlines with filled land and lakes
ax1.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black')
ax1.add_feature(cartopy.feature.LAKES)
ax1.gridlines(xlocs=np.arange(-180.,181.,60.), ylocs=np.arange(-90.,91.,30.))
# add title
ax1.set_title('Ground Tracks')
# stronger linewidth on frame
ax1.spines['geo'].set_linewidth(2.0)
ax1.spines['geo'].set_capstyle('projecting')
# adjust subplot within figure
f1.subplots_adjust(left=0.02,right=0.98,bottom=0.05,top=0.98)

# output plot of fit height for each segment
spots = rsps['spot'].unique()
f2, ax2 = plt.subplots(num=2,ncols=len(spots),sharey=True,figsize=(8,6))
for j,spot in enumerate(spots):
    rsps_per_spot = rsps[rsps["spot"]==spot]
    ax2[j].plot(rsps_per_spot['segment_id'],rsps_per_spot['h_mean'],'r')
    ax2[j].set_xlabel('Segment ID')
    ax2[j].set_title('Spot {0:d}'.format(spot))
ax2[0].set_ylabel('Along Track Elevation [m]')
title = 'SlideRule ATL06 Segment Fits\nSegment Length {0:d}m, Step Size {1:d}m'
f2.suptitle(title.format(LengthSlider.value,StepSlider.value))
f2.subplots_adjust(wspace=0.05)

# show plots
plt.show()